In [8]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random

In [9]:
import json
with open('intents.json') as file:
    data = json.load(file)

In [10]:
words = []
labels = []
docs_x = []
docs_y = []

In [11]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [12]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [13]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [14]:
training = numpy.array(training)
output = numpy.array(output)

In [15]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [16]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 4999  | total loss: 0.08984 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.08984 - acc: 0.9942 -- iter: 32/33
Training Step: 5000  | total loss: 0.08259 | time: 0.012s
| Adam | epoch: 1000 | loss: 0.08259 - acc: 0.9948 -- iter: 33/33
--
INFO:tensorflow:C:\Users\HP\Downloads\StudyMaterials\AI chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:C:\Users\HP\Downloads\StudyMaterials\AI chatbot\model.tflearn.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:C:\Users\HP\Downloads\StudyMaterials\AI chatbot\model.tflearn.index
INFO:tensorflow:0
INFO:tensorflow:C:\Users\HP\Downloads\StudyMaterials\AI chatbot\model.tflearn.meta
INFO:tensorflow:100


In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type get lost to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "get lost":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type get lost to stop)!
